In [51]:
import numpy as np
import pandas as pd

from dask import dataframe as dd
import time
import os
from dask.distributed import Client

fname = '311_Service_Requests_from_2010_to_Present.csv'

In [66]:
# rows_to_keep = [::100, :]
print(os.path.getsize("311_short.csv")/1024/1024/1024 , "GB")

# df = pd.read_csv("311_Service_Requests_from_2010_to_Present.csv", skiprows = lambda x: x not in [::100])

0.7832227107137442 GB


In [52]:
client = Client(processes=False)
client

/Users/epavlov/.pyenv/versions/3.7.4/lib/python3.7/site-packages/distributed/node.py:161: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 58129 instead
  f"Port {expected} is already in use.\n"


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://192.168.1.179:58129/status,
Dashboard: http://192.168.1.179:58129/status,Workers: 1
Total threads: 8,Total memory: 16.00 GiB
Status: running,Using processes: False
Comm: inproc://192.168.1.179/27467/21,Workers: 1
Dashboard: http://192.168.1.179:58129/status,Total threads: 8
Started: Just now,Total memory: 16.00 GiB
Comm: inproc://192.168.1.179/27467/24,Total threads: 8
Dashboard: http://192.168.1.179:58130/status,Memory: 16.00 GiB
Nanny: None,


In [49]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [168]:
# could not read 7GB file using pandas, found package 'dask' which provides multithreaded capabilities
start = time.time()
# cols_to_read = [0,1,2,3,5,8,16,19,20,23,24,25]
cols_to_read = [0,1,5,8,16,25]
# dates = ['Created Date'] 
# types = {'Unique Key':object, 'Incident Zip':object}
types = {'Unique Key':object, 'Incident Zip':object}
dask_df = dd.read_csv(fname, usecols=cols_to_read, dtype = types)
end = time.time()
print("Read csv with dask: ",(end-start),"sec")
dask_df = dask_df.compute(scheduler='threads')
end1 = time.time()
print("Computed csv with dask: ",(end1-end),"sec")

Read csv with dask:  0.04510998725891113 sec
Computed csv with dask:  47.33354711532593 sec


In [169]:
dask_df.dtypes


Unique Key        object
Created Date      object
Complaint Type    object
Incident Zip      object
City              object
Borough           object
dtype: object

In [170]:
dask_df.head()

,Unique Key,Created Date,Complaint Type,Incident Zip,City,Borough
0,25804767,06/24/2013 11:47:39 AM,Illegal Parking,10306,STATEN ISLAND,STATEN ISLAND
1,25804768,06/24/2013 07:06:14 AM,Illegal Parking,10065,NEW YORK,MANHATTAN
2,25804769,06/24/2013 10:16:53 AM,Illegal Parking,11236,BROOKLYN,BROOKLYN
3,25804770,06/24/2013 04:31:58 PM,Traffic,11367,FLUSHING,QUEENS
4,25804774,06/24/2013 09:26:03 AM,Overgrown Tree/Branches,11214,BROOKLYN,BROOKLYN


In [171]:
# dask_df['Complaint Type'].unique()

In [172]:
dask_df.to_csv("311_short.csv")

In [173]:
dask_df['Created Date'] = pd.to_datetime(dask_df['Created Date'])

In [82]:
df1 = dask_df.groupby(['Complaint Type']).filter(lambda x : len(x)>5000)

In [77]:
df1.shape

(12026993, 5)

In [78]:
dask_df.shape

(12040777, 5)

In [79]:
dask_df.groupby(['Complaint Type']).count()

,Created Date,Incident Zip,City,Borough
Complaint Type,,,,
AGENCY,7,7,7,7
APPLIANCE,61345,61343,61343,61345
ATF,168,168,168,168
Adopt-A-Basket,388,388,388,388
Advocate - Levy,2,2,2,2
...,...,...,...,...
Water System,377586,377542,377569,377584
Window Guard,209,209,209,209
X-Ray Machine/Equipment,52,52,52,52


In [83]:
df1.groupby(['Complaint Type']).count()

,Created Date,Incident Zip,City,Borough
Complaint Type,,,,
APPLIANCE,61345,61343,61343,61345
Air Quality,44888,44860,44861,44888
Animal Abuse,51183,51176,51178,51183
Asbestos,11708,11705,11705,11708
BEST/Site Safety,14964,14964,14964,14964
...,...,...,...,...
Vending,24604,24601,24601,24604
WATER LEAK,171192,171186,171186,171192
Water Conservation,25851,25850,25851,25851


In [84]:
df1['Complaint Type'].unique()

array(['Illegal Parking', 'Traffic', 'Overgrown Tree/Branches',
       'Illegal Tree Damage', 'Traffic Signal Condition',
       'Taxi Complaint', 'Noise - Commercial', 'Noise - Residential',
       'Consumer Complaint', 'Maintenance or Facility',
       'Street Light Condition', 'Food Poisoning', 'Derelict Vehicle',
       'Indoor Air Quality', 'Noise - Street/Sidewalk', 'Noise - Park',
       'Non-Emergency Police Matter', 'Rodent', 'Noise - Vehicle',
       'Blocked Driveway', 'Drinking', 'Vending', 'Food Establishment',
       'Damaged Tree', 'Dead Tree', 'For Hire Vehicle Complaint',
       'School Maintenance', 'Water System', 'BEST/Site Safety',
       'General Construction/Plumbing', 'Unsanitary Animal Pvt Property',
       'Sidewalk Condition', 'Air Quality', 'Noise', 'Broken Muni Meter',
       'Standing Water', 'Lead', 'Street Condition',
       'Hazardous Materials', 'Sewer', 'Water Conservation',
       'Housing - Low Income Senior', 'Street Sign - Dangling',
       'Stree

In [85]:
df1['Created Date'] = pd.to_datetime(df1['Created Date'])

In [87]:
df1.dtypes

Created Date      datetime64[ns]
Complaint Type            object
Incident Zip              object
City                      object
Borough                   object
dtype: object

In [88]:
df1.head()


,Created Date,Complaint Type,Incident Zip,City,Borough
0,2013-06-24 11:47:39,Illegal Parking,10306,STATEN ISLAND,STATEN ISLAND
1,2013-06-24 07:06:14,Illegal Parking,10065,NEW YORK,MANHATTAN
2,2013-06-24 10:16:53,Illegal Parking,11236,BROOKLYN,BROOKLYN
3,2013-06-24 16:31:58,Traffic,11367,FLUSHING,QUEENS
4,2013-06-24 09:26:03,Overgrown Tree/Branches,11214,BROOKLYN,BROOKLYN


In [89]:
df1.sort_values('Created Date', inplace=True)

In [90]:
df_copy = df1.copy()

In [132]:
df1 = df_copy.copy()

In [144]:
df1.index.name = 'id'

In [121]:
df1.set_index('Created Date', inplace=True)

In [137]:
df1.reset_index()

,id,Created Date,Complaint Type,Incident Zip,City,Borough
0,85662,2010-01-04 08:28:15,BEST/Site Safety,10304,STATEN ISLAND,STATEN ISLAND
1,101729,2010-01-04 11:38:57,Building/Use,10314,STATEN ISLAND,STATEN ISLAND
2,87260,2010-01-04 12:23:05,Building/Use,11234,BROOKLYN,BROOKLYN
3,106217,2010-01-04 12:32:57,BEST/Site Safety,10466,BRONX,BRONX
4,111485,2010-01-04 14:10:44,Building/Use,10306,STATEN ISLAND,STATEN ISLAND
...,...,...,...,...,...,...
11936070,19257,2019-05-08 01:58:33,Noise - Residential,11212,BROOKLYN,BROOKLYN
11936071,17162,2019-05-08 02:12:19,Blocked Driveway,11236,BROOKLYN,BROOKLYN
11936072,18236,2019-05-08 02:12:46,Noise - Street/Sidewalk,11216,BROOKLYN,BROOKLYN
11936073,19066,2019-05-08 02:14:25,Noise - Vehicle,11207,BROOKLYN,BROOKLYN


In [138]:
df1

,Created Date,Complaint Type,Incident Zip,City,Borough
id,,,,,
85662,2010-01-04 08:28:15,BEST/Site Safety,10304,STATEN ISLAND,STATEN ISLAND
101729,2010-01-04 11:38:57,Building/Use,10314,STATEN ISLAND,STATEN ISLAND
87260,2010-01-04 12:23:05,Building/Use,11234,BROOKLYN,BROOKLYN
106217,2010-01-04 12:32:57,BEST/Site Safety,10466,BRONX,BRONX
111485,2010-01-04 14:10:44,Building/Use,10306,STATEN ISLAND,STATEN ISLAND
...,...,...,...,...,...
19257,2019-05-08 01:58:33,Noise - Residential,11212,BROOKLYN,BROOKLYN
17162,2019-05-08 02:12:19,Blocked Driveway,11236,BROOKLYN,BROOKLYN
18236,2019-05-08 02:12:46,Noise - Street/Sidewalk,11216,BROOKLYN,BROOKLYN


In [164]:
# count_df = df1[['Creation Date','Complaint Type','Incident Zip']].groupby([pd.Grouper(key='Created Date', freq='Y'),'Complaint Type','Incident Zip'])['id'].count().reset_index(name="count")
count_df = df1.groupby(by = ['Complaint Type', 'Incident Zip', pd.Grouper(key='Created Date', freq='Y')]).count()

In [165]:
count_df.to_csv("counted_complaints.csv")

In [166]:
count_df

City  Borough
Complaint Type Incident Zip Created Date               
APPLIANCE      10001        2014-12-31       4        4
                            2015-12-31      17       17
                            2016-12-31      12       12
                            2017-12-31      13       13
                            2018-12-31      10       10
...                                        ...      ...
Water System   11697        2014-12-31       4        4
                            2015-12-31       7        7
                            2016-12-31       5        5
                            2017-12-31      12       12
                            2018-12-31       4        4

[118733 rows x 2 columns]